[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/matog/Flacso_ciencia_de_datos_python_2022/blob/main/Clase4/5%20-%20GIS.ipynb)


# GIS con GEOPANDAS 

https://www.ign.gob.ar/NuestrasActividades/InformacionGeoespacial/CapasSIG

Intro a sistemas de coordenadas y proyecciones: 
- [Video](https://www.youtube.com/watch?v=mNG2vIBs7bU) 
- [ArcGis](https://pro.arcgis.com/es/pro-app/latest/help/mapping/properties/coordinate-systems-and-projections.htm)
- [MappingGis](https://mappinggis.com/2022/02/diferencias-entre-los-sistemas-de-coordenadas-geograficas-y-proyectadas/)


### Carga de módulos

In [ ]:
# Librerias necesarias para librerías de goepython
!apt install gdal-bin python-gdal python3-gdal 
# Instalamos rtree (requerimiento de Geopandas)
!apt install python3-rtree 
# Instalamos Geopandas desde el repositorio, para tomar la última versión
!pip install git+git://github.com/geopandas/geopandas.git
# !pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

Vamos a trabajar con varios dataframe espaciales (GDF). Son dataframes que contienen información espacial, ya sea de un punto, una línea o un polígono. Esta información está contenida en la columna `geometry` de los GDF.

La carga es similar a un dataframe, con el comando `.read_file`.

In [ ]:
# Departamentos de Argentina
gdf_depart = gpd.read_file('https://github.com/matog/Flacso_ciencia_de_datos_python_2022/blob/main/Clase4/data/departamento.zip?raw=true')
# Limites provinciales
gdf_prov = gpd.read_file('https://github.com/matog/Flacso_ciencia_de_datos_python_2022/blob/main/Clase4/data/provincia.zip?raw=true')

### Geopandas Atributos 

    - area: superficie (en unidades de la proyección, ver https://geopandas.org/en/stable/docs/user_guide/projections.html)
    - bounds: tupla de las coordinadas max y min de cada eje
    - total_bounds: tupla de coordinadas max y min en cada eje, para el total de la geoserie
    - geom_type: tipo de geometría.
    - is_valid: testea si las coordinadsa forman una figura qeu es una figura geométrica razonable (de acuerdo a https://www.ogc.org/standards/sfa)

In [ ]:
gdf_prov.loc[gdf_prov['nam'] == 'Catamarca'].is_valid

### Geopandas Métodos:

    - distance(): Devuelve una serie con la distancia mínima entre cada registro
    - centroid: devuelve una GeoSeries de centroides
    - representative_point(): devuelve una GeoSeries con puntos que se encuentran dentro de cada geometria. No devuelve centroides.
    - to_crs(): cambia el sistema de coordenadsa
    - plot(): Grafica la geoserie.

In [ ]:
# uni['distancia del anterio'] = uni.distance(uni.shift(1))

In [ ]:
gdf_prov.centroid

Vamos a intentar graficar un mapa [coropletico](https://es.wikipedia.org/wiki/Mapa_coropl%C3%A9tico), con la población 2010 de los departamentos de la provincia de Mendoza.

Mas arriba en este notebook cargamos los departamentos de toda la Argentina. Vamos a quedarnos solo con los de Mendoza

In [ ]:
gdf_mendoza = gdf_depart.loc[gdf_depart['fdc'] == 'IDE Mendoza']

In [ ]:
plt.figure(figsize=(20,12))
gdf_mendoza.plot();

In [ ]:
tables=pd.read_html('https://es.wikipedia.org/wiki/Anexo:Municipios_de_la_provincia_de_Mendoza')

In [ ]:
type(tables)

In [ ]:
tables[0]

In [ ]:
table = tables[0]

In [ ]:
table.rename(columns = {
                      'Municipio (departamento)': 'departamento',
                      'Población (2010)[4]​': 'poblacion_2010',
                      'Superficie (km²)': 'Superficie_km2', 
                      'Densidad (hab./km²)':'densidad',
                      'Distritos':'distritos',
                      'Número de concejales':'concejales'}, 
                inplace = True)


In [ ]:
table.columns

In [ ]:
table = table[['departamento', 'poblacion_2010', 'Superficie_km2', 'densidad', 'distritos', 'concejales']]

In [ ]:
gdf_mendoza = pd.merge(gdf_mendoza, 
               table,
               left_on = 'nam',
               right_on = 'departamento',
               how= 'left')

In [ ]:
gdf_mendoza.loc[gdf_mendoza['departamento'].isna()]

In [ ]:
table.loc[table['departamento'] == 'Mendoza (municipio)/ Capital (departamento)' , 'departamento'] = 'Capital'

In [ ]:
gdf_mendoza.columns

In [ ]:
gdf_mendoza = gdf_mendoza[['gid', 'nam', 'geometry','poblacion_2010', 'Superficie_km2', 'densidad',
       'distritos', 'concejales']]

In [ ]:
gdf_mendoza.plot(column = 'poblacion_2010',
         legend = True,
         figsize=(10, 10),
         cmap='YlGnBu',
        );

In [ ]:
# Colormaps = https://matplotlib.org/stable/tutorials/colors/colormaps.html# plt.set_cmap('jet')
fig, ax = plt.subplots(figsize=(10, 10))

gdf_mendoza.plot(column = 'poblacion_2010',
             legend = True,
             cmap='YlGnBu',
             ax = ax
            );
ax.set(title='Mapa coroplético de la  población por departamento - Mendoza (2010)')

ax.set_axis_off()
plt.show()

#### Localidades censales 

https://datos.gob.ar/ar/dataset/jgm-servicio-normalizacion-datos-geograficos/archivo/jgm_8.9

Cargamos un csv con localidades censales. No es GDF, a pesar que contiene columnas con latitud y longitud, son columnas de un dataframe estandar.

In [ ]:
localidades = pd.read_csv('https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.29/download/localidades-censales.csv')

In [ ]:
localidades.head(3)

Nos quedamos sólo con los registros de Mendoza

In [ ]:
localidades = localidades.loc[localidades['provincia_nombre']=='Mendoza']

Y con tres columnas. La latitud y longitud del centroide de la localidad, y su nombre.

In [ ]:
localidades = localidades[['centroide_lat','centroide_lon', 'nombre']]

In [ ]:
localidades.head(3)

Ahora convertimos el dataframe tradicional en un GDF, indicandole que columna indica la latitud, y cual la longitud.

In [ ]:
gdf_localidades = gpd.GeoDataFrame(localidades, 
                                   geometry=gpd.points_from_xy(localidades['centroide_lon'], 
                                                               localidades['centroide_lat']),
                                   crs='EPSG:4326')

In [ ]:
gdf_localidades.head(3)

In [ ]:
gdf_localidades.plot()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

gdf_mendoza.plot(ax = ax);

gdf_localidades.plot(marker='o', color='red', markersize=15, ax = ax);

ax.set(title='Localidades Censales - Mendoza (2010)')

# Eliminamos los ejes
ax.set_axis_off()
plt.show()

### Contextily

http://darribas.org/gds19/content/labs/lab_03.html

In [ ]:
!pip install contextily

In [ ]:
import contextily as cx

In [ ]:
gdf_mendoza_crs = gdf_mendoza.to_crs(epsg=3857)
gdf_loc = gdf_localidades.to_crs(epsg=3857)

In [ ]:
gdf_mendoza

In [ ]:
# Filtramos departamentos del Gran Mendoza: Capital, Godoy Cruz, Guaymallén , Las Heras, Lavalle, Luján y Maipú
gdf_dep_gm = gdf_mendoza.loc[(gdf_mendoza['nam']=='Las Heras') | 
                             (gdf_mendoza['nam']=='Capital') | 
                             (gdf_mendoza['nam']=='Godoy Cruz') |
                             (gdf_mendoza['nam']=='Guaymallén') |
                             (gdf_mendoza['nam']=='Lavalle') |
                             (gdf_mendoza['nam']=='Luján de Cuyo') |
                             (gdf_mendoza['nam']=='Maipú')]




In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
gdf_mendoza.plot(ax = ax, alpha=0.5)
gdf_dep_gm.plot(ax=ax, facecolor = 'green', alpha = 0.5)
gdf_localidades.plot(alpha=0.3, marker='o', color='red', markersize=15, ax=ax)
cx.add_basemap(ax, crs=gdf_localidades.crs)
ax.set(title='Localidades Censales - Mendoza (2010) - Resaltado: Gran Mendoza');
ax.set_axis_off()


### Geocoding

[Documentacion](https://automating-gis-processes.github.io/site/notebooks/L3/geocoding_in_geopandas.html)

Que ocurre si no tenés la latitud y la longitud de los puntos que queremos marcar, pero si su dirección postal?

Podemos utilizar algún servicio de _geocodificación_, al cual le pasamos las direcciones lo mas completas posibles, y nos devuelve la latitud y la longitud del punto.

Geopandas tiene una herramienta para eso, que permite utilizar varios servicios como nominatim, google, bing, yahoo, y openmapquest.

Nosotrs vamos a utilizar el servicio de `nominatim` ([link](https://nominatim.org)), que se basa en [OpenStreetMap](https://www.openstreetmap.org) para geocodificar.

Existen otros paquete que prestan servicios similares, con una mayor cantidad de proveedores gratuitos o pagos, como [geocoder](https://geocoder.readthedocs.io/)

In [ ]:
from geopandas.tools import geocode

In [ ]:
tabla = pd.read_html('https://es.wikipedia.org/wiki/Anexo:Universidades_nacionales_de_Argentina', header =1)

In [ ]:
universidades  = tabla[0]

In [ ]:
universidades

In [ ]:
universidades['addr'] = universidades['Dirección'] + ', ' + universidades['Ciudad'] + ', ' + universidades['Provincia'] + ', Argentina'

In [ ]:
universidades.columns

In [ ]:
universidades = universidades[['Nombre', 'Acrónimo', 'addr']]

In [ ]:
universidades.head(3)

Geocodificamos las direcciones con Nominatim. En el parámetro `user_agent` se debería poner el nombre de la app a modo de descripción.

In [ ]:
geo = geocode(universidades['addr'], provider='nominatim', user_agent='test_flacso', timeout=4)

In [ ]:
geo.head(3)

"al costado" de la base de universidades, le pegamos los parametros geolocalizados mediante un `join`

In [ ]:
gdf_uni = geo.join(universidades)

In [ ]:
type(gdf_uni)

Graficamos las universidades:

In [ ]:
gdf_uni.plot();

Vamos a graficar la capa universidades arriba de la capa de límites provinciales.

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
gdf_prov.plot(ax = ax, alpha=0.5)
# dep.plot(ax=ax, facecolor = 'green', alpha = 0.5)
gdf_uni.plot(alpha=0.3, marker='o', color='red', markersize=15, ax=ax)

ax.set_axis_off();

Claramente un punto no se logró geocodificar correctamente, devolviendo valores erroneos de lat y lon para la dirección registrada. Existe algunas formas de lidiar con esto:

#### **Acotando el mapa a la superficie del mapa de Argentina (es todo el cuadro, no sólo el territorio):**

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))

# Graficamos las provincias
gdf_prov.plot(ax = ax, alpha=0.5)

# Graficamos las universidades
gdf_uni.plot(alpha=0.3, marker='o', color='red', markersize=15, ax=ax)

# Titulo
ax.set(title='Localidades Censales - Mendoza (2010)')

# Limites del mapa de provincias
ax.set_ylim(gdf_prov.bounds.miny.min(), gdf_prov.bounds.maxy.max())
ax.set_xlim(gdf_prov.bounds.minx.min(), gdf_prov.bounds.maxx.max())

# Borramos los ejes
ax.set_axis_off();

El punto que estaba fuera de Argentina sigue estando en el geodataframe de las universidades, pero no lo mostramos porque está fuera de los límites del mapa de provincias. Para tener una solución definitiva deberíamos eliminarlo del GDF. 
Podemos hacerlo teniendo en cuenta que su latitud y su longitud están por afuera de los límites del mapa de Argentina

In [ ]:
# Limites del mapa de Argentina
prov_miny = gdf_prov.bounds.miny.min()
prov_maxy = gdf_prov.bounds.maxy.max()
prov_minx = gdf_prov.bounds.minx.min()
prov_maxx = gdf_prov.bounds.maxx.max()

In [ ]:
# Nos quedamos con las universidades geocodificadas, incluso con las que están fuera de los límites. 
# Pero eliminamos las que devolvieron Nan
gdf_uni = gdf_uni.loc[gdf_uni['address'].notna()]

In [ ]:
# Creamos un campo lon y lat para cada universidad, para comparar con los limites del mapa.
# Simplemente tomamos el valor x e y de la columna geometry (esto es posible porque es un geodataframe)
gdf_uni['lon'] = gdf_uni['geometry'].x
gdf_uni['lat'] = gdf_uni['geometry'].y

In [ ]:
# Filtramos los que tienen una longitud mayor al límite
gdf_uni.loc[gdf_uni['lon']>prov_maxx]

In [ ]:
# Nos quedamos con el resto de los puntos, salvo los dos que están fuera del límite..
gdf_uni = gdf_uni.loc[gdf_uni['lon']<prov_maxx]

In [ ]:
# Guardamos el GDF de las universidades como GDF (no como DF)
gdf_uni.to_file('data/uni.geojson', driver='GeoJSON')  

In [ ]:
f, ax = plt.subplots(1, figsize=(30, 20))

# Graficamos las provincias
gdf_prov.plot(ax = ax, alpha=0.5)

# Graficamos las universidades
gdf_uni.plot(alpha=0.3, marker='o', color='red', markersize=15, ax=ax)

# Titulo
ax.set(title='Universidades Nacionales')

# Marcamos cada punto con su acrónimo
for x, y, label in zip(gdf_uni['geometry'].x, gdf_uni['geometry'].y, gdf_uni['Acrónimo']):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")


# Borramos los ejes
ax.set_axis_off();

### Buffers

El análisis de buffers es una importante tarea en el geoprocesamiento. Se utiliza para obtener una distancia alrededor de un punto. 

En este ejemplo, crearemos un buffer alrededor de cada universidad, como proxy de la zona de influencia.

Es necesario saber qué CRS/proyección se está utilizando para obtener el resultado correcto. Necesitamos usar una proyección que mida en metros, no en grados ni en ninguna otra alternativa. Este es un error clásico en el mundo de Geodata.

En una primera aproximación, vamos a seguir trabajando con los puntos localizados en Mendoza Capital.

In [ ]:
gdf_uni = gpd.read_file('https://raw.githubusercontent.com/matog/Flacso_ciencia_de_datos_python_2022/main/Clase4/data/uni.geojson')

In [ ]:
gdf_caba = gpd.read_file('https://raw.githubusercontent.com/mgaitan/departamentos_argentina/master/departamentos-ciudad_autonoma_de_buenos_aires.json')

In [ ]:
gdf_caba.plot()

In [ ]:
gdf_uba = gdf_uni.loc[gdf_uni['Acrónimo']=='UBA']

In [ ]:
gdf_uba.crs = {"init": "EPSG:4326"}
gdf_caba.crs = {"init": "EPSG:4326"}
gdf_uba = gdf_uba.to_crs(epsg = 5343)
gdf_caba = gdf_caba.to_crs(epsg = 5343)

Podemos crear un buffer para todos los registros del GDF (aunque en este caso tenemos solo la UBA), creando una columna para ese fin. El buffer se mide en unidades de medida de del sistema de coordenadas que estemos utilizando en la capa:

In [ ]:
gdf_uba['buffer'] = gdf_uba['geometry'].buffer(1500)

O podemos crear varios buffer individuales para el mismo punto:

In [ ]:
uba_300 = gdf_uba.buffer(300)
uba_800 = gdf_uba.buffer(800)
uba_1000 = gdf_uba.buffer(1000)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

gdf_caba.plot(ax=ax)

gdf_uba['buffer'].plot(ax=ax, color='lightblue')
gdf_uba.plot(ax=ax, color='yellow', markersize=10); # Universidad

ax.set(title='Buffer en columna')

ax.set_axis_off();

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

gdf_caba.plot(ax=ax)

# Ordenas los buffers de los mas amplios al los mas pequeños, para que los grandes no tapen a los chicos. 
# O ajustar el alpha, que genera transparencia
uba_1000.plot(ax=ax, color='red')
uba_800.plot(ax=ax, color='green', alpha = 0.3)
uba_300.plot(ax=ax, color='gray', alpha = 0.3) # Este buffer es tapado por el punto amarillo de la UBA

gdf_uba.plot(ax=ax, color='yellow', markersize=3); # Universidad
ax.set(title='Buffers individuales')

ax.set_axis_off();

Nos quedamos con las provincias de Mendoza, San Juan, San Luis y La Rioja, sólo a efectos prácticos y para mejorar la visualización de la información

In [ ]:
gdf_prov_sel = gdf_prov.loc[(gdf_prov['nam']=='Mendoza') | 
                    (gdf_prov['nam']=='San Juan') | 
                    (gdf_prov['nam']=='San Luis') | 
                    (gdf_prov['nam']=='La Rioja')]

Acá viene un paso clave. Tenemos que convertir los sistemas de coordenadas de referencias a uno que mida distancias en metros (no en grados, ni ninguna otra medida), para poder definir el buffer en metros. Para eso, primero con `init` definimos un sistema de inicio, y con `to_crs`  lo convertimos al epsg 5343, que mide distancias en metros.
Esto debemos correrlos una sola vez, porque si lo corremos dos veces, borramos la información de `geometry`. Si asi ocurriese, volvemos a cargar el GDF.

In [ ]:
gdf_uni.crs = {"init": "EPSG:4326"}
gdf_prov_sel.crs = {"init": "EPSG:4326"}
gdf_uni = gdf_uni.to_crs(epsg = 5343)
gdf_prov_sel = gdf_prov_sel.to_crs(epsg = 5343)

In [ ]:
# Chequeamos que la información de geometry esté ok
gdf_uni.head(3)

Como queremos generar un buffer para cada universidad, generemos una columna.

In [ ]:
gdf_uni['buffer'] = gdf_uni['geometry'].buffer(30000)

In [ ]:
gdf_prov_sel

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))
# cx.add_basemap(ax, crs='epsg:5343')

# prov.plot(ax=ax, color = 'gray')
gdf_prov_sel.plot(ax=ax, color = 'gray')

gdf_uni['buffer'].plot(ax=ax, color='red', cmap='winter')

# Limites del mapa de provincias
ax.set_ylim(gdf_prov_sel.bounds.miny.min(), gdf_prov_sel.bounds.maxy.max())
ax.set_xlim(gdf_prov_sel.bounds.minx.min(), gdf_prov_sel.bounds.maxx.max())

ax.set(title='Buffers para cada universidad en expresado en metros')


# Borramos los ejes
ax.set_axis_off();

Link a documentación de cmpa: https://geopandas.org/en/stable/docs/user_guide/mapping.html

Por mas que hayamos cortado el gráfico según los límites de la capa provincia, quedan algunos puntos que deberíamos borrarlos. Ahora exploraremos operaciones para hacerlo

### Overlays y Spatial Joins 

La principal diferencia entre `spatial join` y `overlay` es que el primer mezcla (_merge_) atributos de otro `geodataframe` a la geometría existente y el segundo, crea una nueva geometría (intersercción, diferencia, etc)

#### Overlays 

https://geopandas.org/en/stable/docs/user_guide/set_operations.html

In [ ]:
gdf_uni_buffer = gpd.GeoDataFrame()
gdf_uni_buffer['geometry'] = gdf_uni['buffer']

In [ ]:
gdf_uni_buffer.head(3)

In [ ]:
overlay_prov_inter = gpd.overlay(gdf_uni_buffer, gdf_prov_sel, how='intersection')

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))

# gdf_prov_sel.plot(ax=ax, color = 'gray', edgecolor='k', cmap='tab10')
overlay_prov_inter.plot(ax = ax, color = 'green')

# Borramos los ejes
ax.set_axis_off();


In [ ]:
overlay_prov_diff = gpd.overlay(gdf_prov_sel, gdf_uni_buffer, how='difference')

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))

overlay_prov_diff.plot(ax = ax, color = 'red')

# Borramos los ejes
ax.set_axis_off();


In [ ]:
gdf_prov_sel_disolve = gdf_prov_sel.dissolve()

In [ ]:
localidades = pd.read_csv('https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.29/download/localidades-censales.csv')

In [ ]:
localidades = localidades[['centroide_lat','centroide_lon', 'nombre']]

In [ ]:
gdf_localidades = gpd.GeoDataFrame(localidades, 
                                   geometry=gpd.points_from_xy(localidades['centroide_lon'], 
                                                               localidades['centroide_lat']),
                                   crs='EPSG:4326')

In [ ]:
gdf_localidades.crs = {"init": "EPSG:4326"}
gdf_localidades = gdf_localidades.to_crs(epsg = 5343)
# prov_sel_disolve.crs = {"init": "EPSG:4326"}
# prov_sel_disolve = prov_sel_disolve.to_crs(epsg = 5343)

In [ ]:
gdf_localidades.head(3)

#### Spatial Joins 

In [ ]:
s_join = gdf_localidades.sjoin(gdf_prov_sel_disolve, how="inner", predicate='intersects')

In [ ]:
s_join

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))

gdf_prov_sel_disolve.plot(ax=ax, color = 'gray', alpha = 0.6)
s_join.plot(ax=ax, color = 'red', alpha = 0.5)

# Borramos los ejes
ax.set_axis_off();

Ahora vamos a pegarle a estos puntos que están en mendoza, el nombre del departamento. Ya está en el GDF esa info, por lo que la vamos a borrar, y buscar en la capa de departamentos con un `spatial_join`.

In [ ]:
gdf_depart = gpd.read_file('https://github.com/matog/Flacso_ciencia_de_datos_python_2022/blob/main/Clase4/data/departamento.zip?raw=true')

In [ ]:
gdf_depart.crs = {"init": "EPSG:4326"}
gdf_depart = gdf_depart.to_crs(epsg = 5343)

Nos quedamos solo con el campo geometry, que tiene la lat y lon de cada punto. A ese campo, le vamos a pegar el resto de la info de la capa de departamentos

In [ ]:
s_join = s_join[['geometry']]

In [ ]:
# Solo quedo la columna necesaria para el spatial join
s_join

In [ ]:
gdf_depar_join = s_join.sjoin(gdf_depart, how="inner", predicate='intersects')

In [ ]:
gdf_depar_join

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))

gdf_depar_join.plot(ax=ax, color = 'gray', alpha = 0.6)
gdf_depart.plot(ax=ax, color = 'yellow', alpha = 0.5)

# Borramos los ejes
ax.set_axis_off();